In [3]:
import pyspark
import time
from pyspark.sql import SQLContext
from pyspark.sql.functions import concat, col, lit, regexp_replace, monotonically_increasing_id, unix_timestamp, from_unixtime
#from pyspark.sql.types import DateType

In [7]:
conf = pyspark.SparkConf()
conf.setAppName("pepXMLtoDisply")
conf.set("spark.executor.memory", "8g").set("spark.executor.cores", "3").set("spark.cores.max", "12")
conf.set("spark.jars.packages", "com.databricks:spark-xml_2.11:0.4.1")
sc = pyspark.SparkContext(conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pepXMLtoDisply, master=spark://10.3.0.128:7077) created by __init__ at <ipython-input-4-83f4520305a3>:5 

In [26]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()
p = figure()
p.line([1,2,3,4,5],[6,7,8,9,10], line_width=2)
show(p)

Loading BokehJS ...

In [9]:
sqlContext=SQLContext(sc)

In [1]:
sc.stop()

NameError: name 'sc' is not defined

In [6]:
inpf='/data/promec/new/mzML/140331_bsa.pep.xml'

In [10]:
df = sqlContext.read.format('com.databricks.spark.xml')\
    .option('attributePrefix','xxx')\
    .options(rowTag='spectrum_query')\
    .load(inpf)
df.rdd.getNumPartitions()
df.printSchema()
#dform.schema
#df = df.repartition(1)

root
 |-- search_result: struct (nullable = true)
 |    |-- search_hit: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- search_score: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |    |-- xxxname: string (nullable = true)
 |    |    |    |    |    |-- xxxvalue: double (nullable = true)
 |    |    |    |-- xxxcalc_neutral_pep_mass: double (nullable = true)
 |    |    |    |-- xxxhit_rank: long (nullable = true)
 |    |    |    |-- xxxmassdiff: double (nullable = true)
 |    |    |    |-- xxxnum_matched_ions: long (nullable = true)
 |    |    |    |-- xxxnum_matched_peptides: long (nullable = true)
 |    |    |    |-- xxxnum_missed_cleavages: long (nullable = true)
 |    |    |    |-- xxxnum_tol_term: long (nullable = true)
 |    |    |    |-- xxxnum_tot_proteins: long (nullable = true)
 |    |    |    |-- xxxpeptid

In [25]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
F1 = udf(lambda x: -x if x <3 else x, StringType())
df.select('xxxassumed_charge', F1('xxxassumed_charge')).take(5)

[Row(xxxassumed_charge=2, <lambda>(xxxassumed_charge)='-2'),
 Row(xxxassumed_charge=3, <lambda>(xxxassumed_charge)='3'),
 Row(xxxassumed_charge=3, <lambda>(xxxassumed_charge)='3'),
 Row(xxxassumed_charge=3, <lambda>(xxxassumed_charge)='3'),
 Row(xxxassumed_charge=3, <lambda>(xxxassumed_charge)='3')]

In [ ]:
#from pyspark.sql.functions import regexp_replace, col
#df.withColumn('@timestamp', regexp_replace(concat(df.attribute_spectrum,lit(timestr)),r'[^\d.]+', '').cast(DateType())).show(5)


In [41]:
selectedData = df.select('xxxassumed_charge').rdd.map(lambda data: Vectors.dense([float(c) for c in data]))
selectedData


PythonRDD[107] at RDD at PythonRDD.scala:48

In [42]:
import sys
import numpy as np
from pyspark.mllib.clustering import KMeans
model = KMeans.train(selectedData, 3)


Py4JJavaError: An error occurred while calling o314.trainKMeansModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 9.0 failed 4 times, most recent failure: Lost task 2.3 in stage 9.0 (TID 31, 192.168.0.57, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-41-da74ef093d12>", line 1, in <lambda>
NameError: name 'Vectors' is not defined

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:2

In [22]:
def parseVector(line):
    return np.array([float(x) for x in line.split(' ')])


    
    
    lines = sc.textFile(sys.argv[1])
    data = lines.map(parseVector)
    k = int(sys.argv[2])
    model = KMeans.train(data, k)
    print("Final centers: str(model.clusterCenters)")


2016-11-05T08:40:36


Py4JJavaError: An error occurred while calling o4911.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 61.0 failed 4 times, most recent failure: Lost task 0.3 in stage 61.0 (TID 144, 192.168.83.91, executor 18): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 220, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 138, in dump_stream
    for obj in iterator:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 209, in _batched
    for item in iterator:


NameError: name 'timestr' is not defined

In [11]:
timestr = time.strftime("%Y%m%d%H%M%S")
outf=inpf+timestr
print(outf)
#dform.toJSON().saveAsTextFile(outf)

/data/promec/new/mzML/140331_bsa.pep.xml20170304221341


AttributeError: 'DataFrame' object has no attribute 'map'

StructType(List(StructField(attribute_assumed_charge,LongType,true),StructField(attribute_end_scan,LongType,true),StructField(attribute_index,LongType,true),StructField(attribute_precursor_neutral_mass,DoubleType,true),StructField(attribute_retention_time_sec,DoubleType,true),StructField(attribute_spectrum,StringType,true),StructField(attribute_spectrumNativeID,StringType,true),StructField(attribute_start_scan,LongType,true),StructField(search_result,StructType(List(StructField(search_hit,ArrayType(StructType(List(StructField(attribute_calc_neutral_pep_mass,DoubleType,true),StructField(attribute_hit_rank,LongType,true),StructField(attribute_massdiff,DoubleType,true),StructField(attribute_num_matched_ions,LongType,true),StructField(attribute_num_matched_peptides,LongType,true),StructField(attribute_num_missed_cleavages,LongType,true),StructField(attribute_num_tol_term,LongType,true),StructField(attribute_num_tot_proteins,LongType,true),StructField(attribute_peptide,StringType,true),Stru

In [232]:
#df.toJSON().first()
#df.toJSON().take(2)
#import org.apache.spark.sql.functions.to_json
#df.select(to_json(struct($"_assumed_charge", $"search_result", $"_precursor_neutral_mass")))
#df.select("_assumed_charge").withColumnRenamed("_assumed_charge", "x1")
#df.replace("_index", "aaa")
#df.limit
#df.withColumn("x7", dfspectra).first()
#df.select('*', (df.attribute_assumed_charge + 10).alias('agePlusTen'))
#df.select('*', (dfspectra).alias('agePlusTen'))
#df.count()
#df.join()
#df.select('attribute_assumed_charge').map(lambda x:(x,1)).take(5)
#from pyspark.sql.functions import concat, col, lit
#df.withColumn('_index', concat(lit('promec'))).show(5)
#df.withColumn('_type', concat(lit('search_hit'))).show(5)
#df.withColumn('_id', concat(df.attribute_spectrum,lit(dfhdr.select("_date").rdd.flatMap(list).first()))).show(5)
#df.withColumn('_id', concat(df.attribute_assumed_charge,dfhdr.select("_date").take(1))).show(5)
#dfhdr.first().getInt(0)
#from pyspark.sql.types import DateType
#import re
#from pyspark.sql.functions import regexp_replace, col
#df.withColumn('@timestamp', regexp_replace(concat(df.attribute_spectrum,lit(timestr)),r'[^\d.]+', '').cast(DateType())).show(5)
#df.withColumn('@timestamp', (concat(df.attribute_spectrum)).cast(DateType())).show(5)
#df.withColumn('@timestamp', regexp_replace(concat(df.attribute_spectrum,lit(timestr)),"[^\d.]+", "").cast(DateType())).show(5)
#df.withColumn('@timestamp', regexp_replace(concat(df.attribute_spectrum,lit(timestr)),"[^\d]+", "").cast(DateType())).show(5)
#df.withColumn('@timestamp', regexp_replace(concat(df.attribute_spectrum,monotonically_increasing_id()),"[^\d]+", "")).show(5)
df.withColumn('@timestamp', concat(lit(timestr),monotonically_increasing_id())).show(5)


+--------------------+-----------------+-----------+--------+-------------------------+---------------------+--------------------+--------------------+-------------+---------------+
|       search_result|xxxassumed_charge|xxxend_scan|xxxindex|xxxprecursor_neutral_mass|xxxretention_time_sec|         xxxspectrum| xxxspectrumNativeID|xxxstart_scan|     @timestamp|
+--------------------+-----------------+-----------+--------+-------------------------+---------------------+--------------------+--------------------+-------------+---------------+
|[WrappedArray([Wr...|                2|          2|       1|              1241.005467|                  0.9|140331_bsa.00002....|controllerType=0 ...|            2|201702231651170|
|[WrappedArray([Wr...|                3|          4|       2|              1495.833029|                  1.6|140331_bsa.00004....|controllerType=0 ...|            4|201702231651171|
|[WrappedArray([Wr...|                3|          7|       3|              1355.595755|   

In [148]:
#dfhdr = sqlContext.read.format('com.databricks.spark.xml').options(rootTag='msms_pipeline_analysis').load(inpf)
dfparam = sqlContext.read.format('com.databricks.spark.xml').options(rootTag='msms_pipeline_analysis', rowTag="search_summary").load(inpf)


In [135]:
dfenz = sqlContext.read.format('com.databricks.spark.xml').options(rootTag='msms_pipeline_analysis', rowTag="sample_enzyme").load(inpf)


In [197]:
dfhdr = sqlContext.read.format('com.databricks.spark.xml').options(rowTag='msms_pipeline_analysis', valueTag='date').load(inpf)


In [396]:
#dfenz
#dfhdr['_date'][1]
#dfhdr.toJSON().first()
#dfhdr.show()
#df.show()
#dfparam.toJSON().take(2)
#dfenz.toJSON().first()
dfhdr.select("_date").rdd.flatMap(list).first()

'2016-10-27T14:13:48'

In [333]:
#selectedData = dfhdr.select("_date").show()
#selectedData = dfhdr.select("msms_run_summary")
#selectedData = dfhdr.select("sample_enzyme")
#df.select("search_result").show()
#dfhdr.select("search_result").first()

DataFrame[msms_run_summary: struct<_base_name:string,_msManufacturer:string,_msModel:string,_raw_data:string,_raw_data_type:string,sample_enzyme:struct<_name:string,specificity:struct<_cut:string,_no_cut:string,_sense:string,date:string>>,search_summary:struct<_base_name:string,_fragment_mass_type:string,_precursor_mass_type:string,_search_engine:string,_search_engine_version:string,_search_id:bigint,enzymatic_search_constraint:struct<_enzyme:string,_max_num_internal_cleavages:bigint,_min_number_termini:bigint,date:string>,parameter:array<struct<_name:string,_value:string,date:string>>,search_database:struct<_local_path:string,_type:string,date:string>>,spectrum_query:array<struct<_assumed_charge:bigint,_end_scan:bigint,_index:bigint,_precursor_neutral_mass:double,_retention_time_sec:double,_spectrum:string,_spectrumNativeID:string,_start_scan:bigint,search_result:struct<search_hit:array<struct<_calc_neutral_pep_mass:double,_hit_rank:bigint,_massdiff:double,_num_matched_ions:bigint,_nu

In [42]:
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.xml').options(rootTag='msms_pipeline_analysis',rowTag='spectrum_query').load('/data/promec/new/mzML/*_bsa.pep.xml')
df.select("search_result").collect()
#df.write().json("/data/promec/new/mzML/bsa.pep.xml.json");
selectedData = df.select("search_result")
#selectedData.write.format("com.databricks.spark.xml").option("rootTag", "msms_pipeline_analysis").option("rowTag", "spectrum_query").save("/data/promec/new/mzML/bsa.pep.xml.json.newbooks.xml")

In [52]:
selectedData = df.select("search_result")
selectedData.printSchema
selectedData=selectedData.toJSON
selectedData.saveAsTextFile("/data/promec/new/mzML/bsa.pep.xml.json")
df.toJSON().first()

AttributeError: 'function' object has no attribute 'saveAsTextFile'

In [58]:
df.toJSON().saveAsTextFile("/data/promec/new/mzML/bsa.pep.xml.json.files4")
#df.toJSON().map(tuple).saveAsTextFile("/data/promec/new/mzML/bsa.pep.xml.json.files2")


In [91]:
df.toJSON().first()

'{"aassumed_charge":2,"aend_scan":2,"aindex":1,"aprecursor_neutral_mass":1241.005467,"aretention_time_sec":0.9,"aspectrum":"140331_bsa.00002.00002.2","aspectrumNativeID":"controllerType=0 controllerNumber=1 scan=2","astart_scan":2,"search_result":{"search_hit":[{"acalc_neutral_pep_mass":1356.569457,"ahit_rank":1,"amassdiff":-115.56399,"anum_matched_ions":2,"anum_matched_peptides":44155946,"anum_missed_cleavages":0,"anum_tol_term":2,"anum_tot_proteins":1,"apeptide":"GGADDPFQEHER","apeptide_next_aa":"G","apeptide_prev_aa":"R","aprotein":"sp|Q01XA0_REVERSED|SYA_SOLUE","atot_num_ions":22,"search_score":[{"aname":"xcorr","avalue":0.563},{"aname":"deltacn","avalue":0.089},{"aname":"deltacnstar","avalue":0.0},{"aname":"spscore","avalue":9.4},{"aname":"sprank","avalue":14.0},{"aname":"expect","avalue":1.74}]},{"acalc_neutral_pep_mass":1897.794113,"ahit_rank":2,"amassdiff":-656.788647,"anum_matched_ions":2,"anum_matched_peptides":44155946,"anum_missed_cleavages":0,"anum_tol_term":2,"anum_tot_pr

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.xml').options(rootTag='msms_pipeline_analysis',rowTag='spectrum_query').load('/data/promec/new/mzML/*_bsa.pep.xml')
df.select("search_result").collect()


In [11]:
import sys

import numpy as np
from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans


def parseVector(line):
    return np.array([float(x) for x in line.split(' ')])


    
    
    lines = sc.textFile(sys.argv[1])
    data = lines.map(parseVector)
    k = int(sys.argv[2])
    model = KMeans.train(data, k)
    print("Final centers: str(model.clusterCenters)")
    sc.stop()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Spark XML, master=spark://10.3.0.128:7077) created by __init__ at <ipython-input-2-d9733dc1196d>:5 

6.0K -rw-r--r--. 1 root root 6.0K Nov 11 20:21 /data/promec/new/mzML/131216_bsa.pep.xml
   0 -rw-r--r--. 1 root root    0 Nov 14 01:15 /data/promec/new/mzML/140122_bsa.pep.xml
 71M -rw-r--r--. 1 root root  71M Nov 12 23:41 /data/promec/new/mzML/140331_bsa.pep.xml
 15M -rw-r--r--. 1 root root  15M Nov 14 01:15 /data/promec/new/mzML/140428_bsa.pep.xml
 26M -rw-r--r--. 1 root root  26M Nov 14 01:15 /data/promec/new/mzML/141210_bsa.pep.xml


In [ ]:
sqlContext = SQLContext(sc)

df = sqlContext.read.format('com.databricks.spark.xml').options(rootTag='msms_pipeline_analysis',rowTag='spectrum_query').load('/data/promec/new/mzML/130716_bsa1.pep.xml')
df.select("search_result").collect()